In [1]:
import json
import os
import re

from vllm import LLM, SamplingParams

# os.environ["CUDA_VISIBLE_DEVICES"] = "1,2"
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
# os.environ["VLLM_LOGGING_LEVEL"] = "DEBUG"
# os.environ["NCCL_P2P_DISABLE"] = "1"

In [2]:
def load_math(path="../datasets/MATH", split="train"):
    with open(os.path.join(path, split, "dataset.json")) as f:
        data = json.load(f)
    
    examples = [{
        'question': q,
        'answer': a,
    } for q, a in zip(data['question'], data['extracted_answers'])]

    return examples

In [3]:
data = load_math(split='test')
# data = load_math(split='train') + load_math(split='test')
len(data)

5000

In [ ]:
llm = LLM(
    # model="meta-llama/Llama-3.1-8B-Instruct", 
    model="hugging-quants/Meta-Llama-3.1-70B-Instruct-AWQ-INT4", 
    tensor_parallel_size=2,
    # download_dir="/home/amittur/.cache/huggingface/hub", 
    max_model_len=10000,
    # gpu_memory_utilization=0.95
    # max_num_seqs=128,
)

INFO 11-27 11:56:14 config.py:350] This model supports multiple tasks: {'embedding', 'generate'}. Defaulting to 'generate'.
INFO 11-27 11:56:14 awq_marlin.py:109] The model is convertible to awq_marlin during runtime. Using awq_marlin kernel.
INFO 11-27 11:56:15 config.py:1020] Defaulting to use mp for distributed inference
INFO 11-27 11:56:15 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='hugging-quants/Meta-Llama-3.1-70B-Instruct-AWQ-INT4', speculative_config=None, tokenizer='hugging-quants/Meta-Llama-3.1-70B-Instruct-AWQ-INT4', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=10000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=awq_marlin, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda,

In [6]:
SOLVE_PROMPT = """
Answer the math problem in the format shown below. End your response with "<|eot_id|>".

---
Problem: <you will be given a math question> 
Reasoning: <your step by step reasoning for the answer>
Answer: <your final answer only>
---

Problem: {}
"""

def get_prompts(data):
    return [SOLVE_PROMPT.format(d['question']) for d in data]

In [7]:
sampling_params = SamplingParams(
    temperature=0,
    max_tokens=5120,
    stop_token_ids=[128001, 128008, 128009],
)

In [8]:
def extract_answer(text):
    match = re.search(r"Answer: (.+)", text, re.DOTALL)
    answer = ''
    reasoning = '[NO_COT_REASONING]'
    if match:
        # reasoning = match.group(1).strip()
        answer = match.group(1).strip()
    return {
        'answer': answer,
        'reasoning': reasoning
    }

def extract_cot_answer(text):
    match = re.search(r"Reasoning: (.+)Answer: (.+)", text, re.DOTALL)
    answer = reasoning = ''
    if match:
        reasoning = match.group(1).strip()
        answer = match.group(2).strip()
    return {
        'answer': answer,
        'reasoning': reasoning
    }

In [9]:
# Sanity Check

outputs = llm.generate(SOLVE_PROMPT.format(data[5]['question']), sampling_params)
print(outputs[0].outputs[0].text)
print(extract_cot_answer(outputs[0].outputs[0].text))

Processed prompts: 100%|██████████| 1/1 [00:06<00:00,  6.70s/it, est. speed input: 26.71 toks/s, output: 30.44 toks/s]

Reasoning: The area of the shaded region is the area of the entire circle minus the area of the unshaded region. The area of the entire circle is $\pi \cdot 6^2 = 36\pi$ square inches. The area of the unshaded region is $\pi \cdot 4^2 = 16\pi$ square inches. Therefore, the area of the shaded region is $36\pi - 16\pi = 20\pi$ square inches. The ratio of the area of the shaded region to the area of the entire circle is $\frac{20\pi}{36\pi} = \frac{5}{9}$. Since the darts land randomly, we can expect $\frac{5}{9}$ of the darts to land in the shaded region. Therefore, we can expect $9 \cdot \frac{4}{9} = 4$ darts to land in the non-shaded region.
Answer: 4
{'answer': '4', 'reasoning': 'The area of the shaded region is the area of the entire circle minus the area of the unshaded region. The area of the entire circle is $\\pi \\cdot 6^2 = 36\\pi$ square inches. The area of the unshaded region is $\\pi \\cdot 4^2 = 16\\pi$ square inches. Therefore, the area of the shaded region is $36\\pi - 1

In [10]:
data_prompts = get_prompts(data)

In [11]:
outputs = llm.generate(data_prompts, sampling_params)
answers = []

for i, output in enumerate(outputs):
    # prompt = output.prompt
    generated_text = output.outputs[0].text

    answers.append(extract_cot_answer(generated_text))

    # print(f"Question Idx: {i}, Response: {generated_text!r}")

Processed prompts:  13%|█▎        | 671/5000 [05:11<34:08,  2.11it/s, est. speed input: 270.40 toks/s, output: 510.89 toks/s]  

WARNING 11-26 12:11:52 scheduler.py:1481] Sequence group 926 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=1


Processed prompts:  15%|█▌        | 769/5000 [06:05<47:30,  1.48it/s, est. speed input: 262.74 toks/s, output: 533.38 toks/s]  

WARNING 11-26 12:12:46 scheduler.py:1481] Sequence group 944 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=51


Processed prompts:  16%|█▋        | 814/5000 [07:19<2:59:53,  2.58s/it, est. speed input: 231.88 toks/s, output: 517.65 toks/s]

...

Processed prompts:  77%|███████▋  | 3861/5000 [1:21:44<07:56,  2.39it/s, est. speed input: 99.83 toks/s, output: 489.03 toks/s]

WARNING 11-26 13:28:29 scheduler.py:1481] Sequence group 4086 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=1801


Processed prompts:  94%|█████████▎| 4680/5000 [1:26:24<02:03,  2.59it/s, est. speed input: 111.18 toks/s, output: 503.98 toks/s]

WARNING 11-26 13:33:04 scheduler.py:1481] Sequence group 4900 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=1851


Processed prompts: 100%|██████████| 5000/5000 [1:36:20<00:00,  1.16s/it, est. speed input: 105.98 toks/s, output: 511.36 toks/s]


In [12]:
# Save answers as json
with open("answers_70b_int4.json", "w") as f:
    json.dump(answers, f)